#### надо установить opencv для python, если не установлен.
#### pip(pip3) install opencv-python

In [2]:
import cv2
import numpy as np
import torch

Пример получения вектора признаков для изображения

In [3]:
# далее задаются параметры HOG-a, они выставлены как в статье.
win_size = (64,128) 
block_size = (16,16)
block_stride = (8,8)
cell_size = (8,8)
nbins = 9
deriv_aperture = 1
win_sigma = -1
histogram_norm_type = 0
l2_hys_threshold = 2.0000000000000001e-01
gamma_correction = 0
nlevels = 64
hog = cv2.HOGDescriptor(win_size,block_size,block_stride,cell_size,nbins,deriv_aperture,win_sigma,
                        histogram_norm_type,l2_hys_threshold,gamma_correction,nlevels)


В базе INRIA размер изображений содержащих пешеходов  - 96х160(train) и 70x134(test). Человек расположен по центру.

Вам нужно получить вектор признаков(дескриптор) для окна 64х128. По умолчанию compute() строит дескрипторы для всех окон размером 64х128 методом скользящего окна, с шагом win_stride по обеим осям. Но если вам нужно получить дескриптор только для одного окна 64х128, то у метода есть аргумент locations, в котором можно передать координаты верхнего левого угла интересующего вас окна (либо список таких координат)

Например, для изображений из train извлечение центрального окна 64х128 выглядит следующим образом:

In [4]:
img = cv2.imread("crop_000010a.png")
height, width = img.shape[:2]
locations = [((width-64)//2, (height-128)//2)]
descriptor = hog.compute(img, locations=locations)
descriptor.size

3780

Изображения фона(отрицательные примеры)  - изображения произвольного размера. В обучающей выборке их 1218. Авторы HOG для обучения выбирали рандомно 10 окон 64х128 из каждого изображения  - всего 12180 примеров, не содержащих пешеходов.

Для этого, например, можно посчитать дескрипторы по всему изображению с win_stride=(4,4) и потом рандомно выбрать 10 из них.

Такие манипуляции вам надо проделать в цикле со всеми изображениями из train: для изображений из папки pos/ выбирать центральное окно, а из папки neg/ - 10 рандомных окон. Собрать их в np.array  или список, как вам будет удобно ну и в файл сохранить тоже.

Для того чтобы получить ответ svm классификатора у класса HOGDescriptor есть метод detect(). Этот метод вам понадобится, чтобы строить кривые качества (DET кривые). Метод detect() работает похожим образом с compute().
Только перед его использованием нужно задать веса SVM:

    hog.setSVMDetector(hog.getDefaultPeopleDetector()) - используется веса из opencv для HOG, обученного на INRIA
В остальном поробуйте сами с detect() разобраться. Если не получится - пишите.

Гайд по построению DET кривых позже добавлю

# Выделим признаки на позитивных картинках

In [5]:
win_stride = (8,8) #шаг скользящего окна в пикселях по ширине и высоте
def imHOG(path_to_image):
    img = cv2.imread(path_to_image)
    height, width = img.shape[:2]
    locations = [((width-64)//2, (height-128)//2)]
    descriptor = hog.compute(img, locations=locations)
    return descriptor

In [6]:
import pandas as pd

images = open("pos.lst")
string = images.read()
images = string.split("\n")
for i in range(len(images)):
    images[i] = images[i][10:]
images = images[:-1]

In [7]:
pos = []
for i in range(len(images)):
    #desc_file.write(str(imHOG(images[i]))+"\n")
    descriptor_current = imHOG(images[i])
    pos.append(descriptor_current)

In [8]:
images = open("pos_test.lst")
string = images.read()
images = string.split("\n")
for i in range(len(images)):
    images[i] = images[i][9:]
images = images[:-1]


In [9]:
pos_test = []
for i in range(len(images)):
    #desc_file.write(str(imHOG(images[i]))+"\n")
    descriptor_current = imHOG(images[i])
    pos_test.append(descriptor_current)

# На негативных

In [10]:
win_stride = (8,8) #шаг скользящего окна в пикселях по ширине и высоте
def imHOG_neg(path_to_image):
    img = cv2.imread(path_to_image)
    win_stride = (4,4)
    #получаем дескрипторы изображения и приводим их к рамеру
    #(кол-во окон на изображении)х(рамер дескриптора одного окна)
    #рамер дексриптора для параметров HOG как в статье - 3780
    descriptors = hog.compute(img, win_stride).reshape(-1,3780)
    indexes = np.random.randint(descriptors.shape[0], size=10)
    ten_random_samples = descriptors[indexes]
    return ten_random_samples

In [11]:
images_neg = open("neg.lst")
string_neg = images_neg.read()
images_neg = string_neg.split("\n")
for i in range(len(images_neg)):
    images_neg[i] = images_neg[i][10:]
images_neg = images_neg[:-1]


In [12]:
neg = []
for i in range(int(len(images_neg))):  #use less neg pictures
    desc_hog = imHOG_neg(images_neg[i])
    for i in range (len(desc_hog)):
        neg.append(desc_hog[i])

In [13]:
images_neg = open("neg_test.lst")
string_neg = images_neg.read()
images_neg = string_neg.split("\n")
for i in range(len(images_neg)):
    images_neg[i] = images_neg[i][9:]
images_neg = images_neg[:-1]

In [14]:
neg_test = []
for i in range(int(len(images_neg))):   #use less neg_test pictures
    desc_hog = imHOG_neg(images_neg[i])
    for descriptor_current in desc_hog:
        neg_test.append(descriptor_current)

# Решейпим полученные дескрипторы pos, pos_test, neg, neg_test

In [15]:
print(len(neg),len(neg_test),len(pos),len(pos_test))

12180 4530 2416 1132


In [16]:
for i in range(len(pos)):
    for j in range(3780):
        pos[i][j] = pos[i][j][0]
        
for i in range(len(pos_test)):
    for j in range(3780):
        pos_test[i][j] = pos_test[i][j][0]

In [17]:
def ReshapeOneHOG(hog):
    new_hog = []
    for i in range(15):
        new_hog.append([])
        new_hog.append([])
        for j in range(7):
            cell = ReshapeCell(hog[i*36*7+36*j : i*36*7+36*(j+1)])
            new_hog[2*i].append(cell[0])
            new_hog[2*i+1].append(cell[1])
            new_hog[2*i].append(cell[2])
            new_hog[2*i+1].append(cell[3])
    return new_hog
            
def ReshapeCell(cell):
    c = []
    c.append(cell[0:9])
    c.append(cell[9:18])
    c.append(cell[18:27])
    c.append(cell[27:36])
    return c

def Reshape_in(Q):
    RES=[]
    for i in range(0,9):
        a=[] #30 array of 14 elem
        for j in range(0,15):
            b=[] #14 elem
            c=[]#14 elem
            for k in range(0,7):
                b.append(Q[2*j*14*9+k*36+i])
                b.append(Q[2*j*14*9+k*36+i+18])
                c.append(Q[2*j*14*9+k*36+i+9])
                c.append(Q[2*j*14*9+k*36+i+27])
            a.append(b)
            a.append(c)
        RES.append(a)
    return RES

In [60]:
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim


class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)    9x30x14
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 3, output channels = 18
        self.conv1 = torch.nn.Conv2d(9, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = torch.nn.Conv2d(18, 36, kernel_size=5, stride=1, padding=1)
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(36 * 14 * 6, 64)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(64, 1)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 32, 32) to (18, 32, 32)   9x30x14 to 18x30x14
        x = F.relu(self.conv1(x))
        #print(x.shape)
        #print(x.shape)
        #to 18x14x6
        x = F.relu(self.conv2(x))   #to 36x14x6
        #print(x.shape)
        #print(x.shape)
        #Size changes from (18, 32, 32) to (18, 16, 16)  36x14x6 to 36x7x3
        x = self.pool(x)
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 16, 16) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        #print(x.shape)
        x = x.view(-1, 36 * 14 * 6)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return F.sigmoid(x)
    
for n in range(3):
    print("Dropout level {}".format(dropout))
    model = SimpleCNN()
    criterion = nn.BCELoss()
    optimizer = optim.Adagrad(model.parameters(), lr=0.001)

    # Train the model
    for i in range(800):
        # Forward pass
        #xinput = torch.tensor(Reshape_in(neg[i]), dtype = torch.float32)
        inputs = []
        labels = []
        for j in range(3):
            inputs.append(Reshape_in(neg[9*i+3*j]))
            labels.append([0])
            inputs.append(Reshape_in(neg[9*i+3*j+1]))
            labels.append([0])
            inputs.append(Reshape_in(neg[9*i+3*j+2]))
            labels.append([0])
            inputs.append(Reshape_in(pos[3*i+j]))
            labels.append([1])
        inputs = torch.from_numpy(np.array(inputs))
        labels = torch.tensor(labels, dtype = torch.float32)
        #print(inputs.shape, labels)

        outputs = model(inputs)
        #print(outputs)

        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 200 == 0:
            print ('Loss: {:.4f}' 
                   .format(loss.item()))
        
    #test
    for h in np.linspace(0.5, 0.7, 4):
        inputs = []
        labels = []
        acc = 0
        for i in range(1100):

            inputs.append(Reshape_in(neg[2*i]))
            labels.append([0])
            inputs.append(Reshape_in(neg[2*i+1]))
            labels.append([0])
            inputs.append(Reshape_in(pos[i]))
            labels.append([1])
        inputs = torch.from_numpy(np.array(inputs))
        labels = torch.tensor(labels, dtype = torch.float32)
        outputs = model(inputs)
        for j in range(len(inputs)):
            if outputs[j] > h:
                if labels[j] == 1:
                    acc += 1
            else:
                if labels[j] == 0:
                    acc += 1
        print("Accuracy {} h {}".format(round(acc/len(labels), 3), round(h,3)))




Dropout level 0.7


/Users/tomininvladislav/env/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Loss: 0.6559
Loss: 0.7484
Loss: 0.0968
Loss: 0.2865
Accuracy 0.862 h 0.5
Accuracy 0.856 h 0.567
Accuracy 0.845 h 0.633
Accuracy 0.825 h 0.7
Dropout level 0.7
Loss: 0.6513
Loss: 0.7478
Loss: 0.1435
Loss: 0.2724
Accuracy 0.865 h 0.5
Accuracy 0.859 h 0.567
Accuracy 0.851 h 0.633
Accuracy 0.83 h 0.7
Dropout level 0.7
Loss: 0.6521
Loss: 0.7134
Loss: 0.1107
Loss: 0.3003
Accuracy 0.864 h 0.5
Accuracy 0.861 h 0.567
Accuracy 0.853 h 0.633
Accuracy 0.834 h 0.7


In [55]:
for h in np.linspace(0.5, 0.7, 4):
    inputs = []
    labels = []
    acc = 0
    for i in range(1100):

        inputs.append(Reshape_in(neg[2*i]))
        labels.append([0])
        inputs.append(Reshape_in(neg[2*i+1]))
        labels.append([0])
        inputs.append(Reshape_in(pos[i]))
        labels.append([1])
    inputs = torch.from_numpy(np.array(inputs))
    labels = torch.tensor(labels, dtype = torch.float32)
    outputs = model(inputs)
    for j in range(len(inputs)):
        if outputs[j] > h:
            if labels[j] == 1:
                acc += 1
        else:
            if labels[j] == 0:
                acc += 1
    print("Accuracy {} h {}".format(round(acc/len(labels), 3), round(h,3)))

/Users/tomininvladislav/env/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Accuracy 0.881 h 0.5
Accuracy 0.884 h 0.567
Accuracy 0.881 h 0.633
Accuracy 0.873 h 0.7
